# BERTopic

In [26]:
from bertopic import BERTopic
from bertopic.representation import KeyBERTInspired
import re

In [27]:
with open('CoRef-TheDaVinciCode.txt', 'r') as f:
    data = f.read()
    data = data.replace("\n", " ")
    data = data.replace("\t", " ")
    data = data.replace("\r", " ")
    data = data.replace("  ", " ")
    data = data.replace("   ", " ")
    data = data.replace("    ", " ")
    data = data.replace("     ", " ")
    ' '.join(data.split())
    data = re.sub('/\s+/', ' ', data)
f.close()
# paragraphs = data.split("CHAPTER")

<>:11: DeprecationWarning: invalid escape sequence \s
<>:11: DeprecationWarning: invalid escape sequence \s
/var/folders/0d/97nbyjxx2cx9d8_vlthd88qh0000gn/T/ipykernel_3404/3908255855.py:11: DeprecationWarning: invalid escape sequence \s
  data = re.sub('/\s+/', ' ', data)


In [28]:
temp = data.split(".")
temp_ = [x for x in temp if x != ['']]
# merge by index of 5 sentences
sentences = [temp_[i:i+5] for i in range(0, len(temp_), 5)]
# merge all subindex of sentences into one string
sentences = [' '.join(x) for x in sentences]
paragraphs = data.split("CHAPTER")

In [29]:
# Fine-tune your topic representations
representation_model = KeyBERTInspired()
topic_model = BERTopic(language="english",representation_model=representation_model, calculate_probabilities=True, verbose=True)
topics, probs = topic_model.fit_transform(paragraphs)

Batches: 100%|██████████| 4/4 [00:03<00:00,  1.11it/s]
2023-05-26 15:03:38,187 - BERTopic - Transformed documents to Embeddings
2023-05-26 15:03:42,103 - BERTopic - Reduced dimensionality
2023-05-26 15:03:42,132 - BERTopic - Clustered reduced embeddings


In [30]:
freq = topic_model.get_topic_info(); freq.head(5)

,Topic,Count,Name
0,-1,6,-1_bank_banks_depository_zurich
1,0,79,0_langdon_langdons_priory_sophie
2,1,24,1_silas_silass_opus_saintsulpice


In [31]:
freq

,Topic,Count,Name
0,-1,6,-1_bank_banks_depository_zurich
1,0,79,0_langdon_langdons_priory_sophie
2,1,24,1_silas_silass_opus_saintsulpice


In [32]:
df = topic_model.get_document_info(paragraphs)
# df only for topic 0
df = df[df['Topic']==0]
# converge all document in df as one single string
text = ' '.join(df['Document'].tolist()).lower()
# count all the occurences of langdon in the text
print('Langdon is present %d times in topic 0' % (text.count('langdon')))

Langdon is present 3486 times in topic 0


In [33]:
df = topic_model.get_document_info(paragraphs)
# df only for topic 1
df = df[df['Topic']==1]
# converge all document in df as one single string
text = ' '.join(df['Document'].tolist()).lower()
# count all the occurences of langdon in the text
print('Langdon is present %d times in topic 1' % (text.count('langdon')))

Langdon is present 99 times in topic 1


In [34]:
df = topic_model.get_document_info(paragraphs)
# df only for topic 0
df = df[df['Topic']==0]
# converge all document in df as one single string
text = ' '.join(df['Document'].tolist()).lower()
# count all the occurences of langdon in the text
print('Silas is present %d times in topic 0' % (text.count('silas')))

Silas is present 130 times in topic 0


In [35]:
df = topic_model.get_document_info(paragraphs)
# df only for topic 1
df = df[df['Topic']==1]
# converge all document in df as one single string
text = ' '.join(df['Document'].tolist()).lower()
# count all the occurences of langdon in the text
print('Silas is present %d times in topic 1' % (text.count('silas')))

Silas is present 782 times in topic 1


In [36]:
topics, probs = topic_model.fit_transform(sentences)

Batches: 100%|██████████| 79/79 [00:25<00:00,  3.04it/s]
2023-05-26 15:04:09,336 - BERTopic - Transformed documents to Embeddings
2023-05-26 15:04:14,033 - BERTopic - Reduced dimensionality
2023-05-26 15:04:14,330 - BERTopic - Clustered reduced embeddings


In [37]:
topic_model.visualize_topics()

In [38]:
topic_model.visualize_heatmap()

In [39]:
import nltk
from nltk.corpus import stopwords
import gensim
from gensim.utils import simple_preprocess
import gensim.corpora as corpora
import pyLDAvis.gensim
import pickle 
import pyLDAvis
import os
from gensim.models.coherencemodel import CoherenceModel

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/kristianvankuijk/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [40]:
def sent_to_words(sentences):
    for sentence in sentences:
        # deacc=True removes punctuations
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))

def remove_stopwords(words):
    stop_words = stopwords.words('english')
    return [word for word in words if word not in stop_words]

In [41]:
stop_words = stopwords.words('english')
sentences = data.split(".")

words = sent_to_words(sentences)
# Remove stop words
words = remove_stopwords(words)

id2word = corpora.Dictionary(words)

In [42]:
topics = topic_model.get_topics()

# Create a dictionary from the words
id2word = corpora.Dictionary(words)

# Create a corpus for coherence evaluation
corpus = [id2word.doc2bow(w) for w in words]

topic_words = [[words for words, _ in topic_model.get_topic(topic)] 
               for topic in range(len(set(topics))-1)]


coherence_model = CoherenceModel(topics=topic_words, texts=words, corpus=corpus, dictionary=id2word, coherence='c_v')
coherence_score = coherence_model.get_coherence()

print("Coherence Score:", coherence_score)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

# LDA

In [44]:
# https://towardsdatascience.com/end-to-end-topic-modeling-in-python-latent-dirichlet-allocation-lda-35ce4ed6b3e0

stop_words = stopwords.words('english')
sentences = data.split(".")

words = sent_to_words(sentences)
# Remove stop words
words = remove_stopwords(words)

id2word = corpora.Dictionary(words)

# Term Document Frequency
corpus = [id2word.doc2bow(word) for word in words]
num_topics = 5

# Build LDA model
lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                       id2word=id2word,
                                       num_topics=num_topics,
                                       passes=20,
                                       iterations=5000)

# Print the keywords in 5 topics
print(lda_model.print_topics())
doc_lda = lda_model[corpus]

# Visualize the topics
pyLDAvis.enable_notebook()

if not os.path.exists('LDA'):
  os.mkdir('LDA')
  
LDAvis_data_filepath = os.path.join('LDA/results')

# Visualize
LDAvis_prepared = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
with open(LDAvis_data_filepath + '10', 'wb') as f:
        pickle.dump(LDAvis_prepared, f)

pyLDAvis.save_html(LDAvis_prepared, 'LDA/results'+ str(10) +'.html')

LDAvis_prepared

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2      0.136093  0.005057       1        1  29.508676
1     -0.014321 -0.054181       2        1  23.169793
4      0.045479  0.006963       3        1  21.636401
3     -0.091407 -0.091031       4        1  13.349142
0     -0.075843  0.133192       5        1  12.335988, topic_info=         Term          Freq         Total Category  logprob  loglift
3956   sophie   2776.000000   2776.000000  Default  30.0000  30.0000
9         the  11504.000000  11504.000000  Default  29.0000  29.0000
410   langdon   3397.000000   3397.000000  Default  28.0000  28.0000
16         of   4383.000000   4383.000000  Default  27.0000  27.0000
2271   bishop    396.000000    396.000000  Default  26.0000  26.0000
...       ...           ...           ...      ...      ...      ...
139       now     87.368810    414.705456   Topic5  -5.4182   0.5352
118      with    108.578879    859.309809   Topic5  -5.2009   0.0240
47         as    104.657844    873.407415   Topic5  -5.2377  -0.0291
111       his     92.887557    644.096683   Topic5  -5.3570   0.1562
95         on     92.784154   1080.586676   Topic5  -5.3581  -0.3623

[358 rows x 6 columns], token_table=      Topic      Freq      Term
term                           
5417      4  0.939899      acre
8827      2  0.045935  airfield
8827      4  0.918709  airfield
1302      3  0.939630   airport
1302      4  0.016201   airport
...     ...       ...       ...
2423      1  0.181508     young
2423      3  0.153584     young
2423      4  0.656221     young
2423      5  0.013962     young
6842      2  0.995554    zurich

[724 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 2, 5, 4, 1])

In [46]:
cm = CoherenceModel(model=lda_model, texts=words, corpus=corpus, coherence='c_v')
coherence = cm.get_coherence()  # get coherence value
print('coherence:', coherence)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 